#Step 1: File completo transaction.csv#

In [ ]:
import pandas as pd
from datetime import datetime, timedelta, date
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division

import plotly.express as px
import plotly.graph_objects as go
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Colab Notebooks/transactions.csv.gz"
df1 = pd.read_csv(path, nrows=100,compression = 'gzip')

Mounted at /content/drive


FileNotFoundError: ignored

In [ ]:
df2=df1.head(5)

Salvo l'head in latex per riferimenti nella tesi

In [ ]:
df2.to_latex("/content/drive/MyDrive/Colab Notebooks/df1.latex")

In [ ]:
df2

,id,chain,dept,category,company,brand,date,productsize,productmeasure,purchasequantity,purchaseamount
0,86246,205,7,707,1078778070,12564,2012-03-02,12.0,OZ,1,7.59
1,86246,205,63,6319,107654575,17876,2012-03-02,64.0,OZ,1,1.59
2,86246,205,97,9753,1022027929,0,2012-03-02,1.0,CT,1,5.99
3,86246,205,25,2509,107996777,31373,2012-03-02,16.0,OZ,1,1.99
4,86246,205,55,5555,107684070,32094,2012-03-02,16.0,OZ,2,10.38


In [ ]:
df1['date'].max(), df1['date'].min()

('2013-07-28', '2012-03-02')

In [ ]:
#Verifico l'intervallo del purchaseamount
df1['purchaseamount'].max(), df1['purchaseamount'].min()

(58658.76, -8593791.0)

In [ ]:
#Cerco gli outlier
fig = px.box(df1,  y="purchaseamount", boxmode="overlay")
fig.show()



In [ ]:
df1.sort_values('purchaseamount',ascending=False)




In [ ]:
#calcolo il purchaseamount per giorno per cercare di capire se vi sono dati mancanti
df1_tot = df1.groupby(['date'])['purchaseamount'].sum().reset_index()
df1_tot

In [ ]:
df1

,date,purchaseamount
0,2012-03-02,7.59
1,2012-03-02,1.59
2,2012-03-02,5.99
3,2012-03-02,1.99
4,2012-03-02,10.38
...,...,...
349655784,2013-07-20,3.00
349655785,2013-07-20,5.49
349655786,2013-07-20,4.00
349655787,2013-07-20,2.79


In [ ]:
#Funzione per controllare le chunk filtrandole per chain=21
def check_chunk(chunck):
  return chunck[chunck.chain==21].copy()


import os
import time
start = time.time()
#Leggere i dati in chunks da 1 milione di righe
chunk = pd.read_csv(os.path.join(path), chunksize=1000000,compression = 'gzip')
end = time.time()

#Creo il dataframe con le righe filtrate e lo comprimo
total_df = pd.DataFrame()
for c in chunk:
  tmp = check_chunk(c)
  total_df = pd.concat([tmp, total_df])

print("Read csv with Chunks: ", (end-start), "sec")

total_df.to_parquet("/content/drive/MyDrive/Colab Notebooks/total_df.parquet")

Read csv with Chunks:  0.009917020797729492 sec


In [ ]:
df_chain = df1.groupby('id').agg({'id': 'count'})

In [ ]:
len(df1['id'].unique())

311541

In [ ]:
df_chain.sort_values('id',ascending= False)

,chain,id
14,21,41074793
40,96,15955983
43,101,13471355
72,214,13411382
57,152,12005477
...,...,...
85,304,4042
100,387,3940
34,81,2432
82,284,171


In [ ]:
#Lista indici da skippare
#indici_skip = df1[df1['chain']!=21].index
indici_skip= df1[df1['chain']!=21].index

In [ ]:
dizionario_indici = { }
dizionario_indici['indici'] = indici_skip

""
chain
2
3
4
6
8
...
522
523
524


In [ ]:
import json #load per caricare, dump per scrivere
json.dump(dizionario_indici)


In [ ]:
#caricho il json "dizionario_indici"
indici = dizionario_indici["indici"]
df1 = pd.read_csv(path, skiprows = indici, compression = 'gzip')

In [ ]:
#Check: lunghezza uguale e chain= 21
#BUON FINE
#1 dati letti
#2 2 cechk ok
#Controllare che le date siano ok
# numero di clienti

In [ ]:
cus_rnd

,id,date
145356638,696521494,2013-03-29
332691691,4627280357,2012-09-01
2481672,104602694,2012-03-16
76601983,496516019,2013-04-11
336765598,4631006750,2012-06-01
...,...,...
71659844,495624320,2012-11-19
121722302,606181124,2012-07-02
201806346,2073149667,2012-05-03
217800036,2409397280,2013-04-05


In [ ]:
df1[df1['id']==106315538]

,id,date
2999901,106315538,2012-03-04
2999902,106315538,2012-03-04
2999903,106315538,2012-03-05
2999904,106315538,2012-03-05
2999905,106315538,2012-03-05
...,...,...
3000506,106315538,2013-06-20
3000507,106315538,2013-06-20
3000508,106315538,2013-06-20
3000509,106315538,2013-06-20


In [ ]:
cus_rnd.groupby('id').count()

,date
id,
86246,1
86252,1
23447825,1
51099044,1
53404104,1
...,...
4741340202,1
4741352144,1
4747697684,1


In [ ]:
from datetime import datetime, timedelta
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division

import plotly.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

#inititate Plotly
pyoff.init_notebook_mode()

#load our data from CSV
tx_data = pd.read_csv('data.csv')

#convert the string date field to datetime
tx_data['InvoiceDate'] = pd.to_datetime(tx_data['InvoiceDate'])



**Recency**

In [ ]:
#create a generic user dataframe to keep CustomerID and new segmentation scores
tx_user = pd.DataFrame(tx_data['CustomerID'].unique())
tx_user.columns = ['CustomerID']

#get the max purchase date for each customer and create a dataframe with it
tx_max_purchase = tx_uk.groupby('CustomerID').InvoiceDate.max().reset_index()
tx_max_purchase.columns = ['CustomerID','MaxPurchaseDate']

#we take our observation point as the max invoice date in our dataset
tx_max_purchase['Recency'] = (tx_max_purchase['MaxPurchaseDate'].max() - tx_max_purchase['MaxPurchaseDate']).dt.days

#merge this dataframe to our new user dataframe
tx_user = pd.merge(tx_user, tx_max_purchase[['CustomerID','Recency']], on='CustomerID')

tx_user.head()

#plot a recency histogram

plot_data = [
    go.Histogram(
        x=tx_user['Recency']
    )
]

plot_layout = go.Layout(
        title='Recency'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)


**Scegliere i gruppi di Recency**

In [ ]:
from sklearn.cluster import KMeans

sse={}
tx_recency = tx_user[['Recency']]
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(tx_recency)
    tx_recency["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_ 
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
#build 4 clusters for recency and add it to dataframe
kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Recency']])
tx_user['RecencyCluster'] = kmeans.predict(tx_user[['Recency']])

#function for ordering cluster numbers
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

tx_user = order_cluster('RecencyCluster', 'Recency',tx_user,False)

**Frequency**

In [ ]:
tx_frequency = tx_uk.groupby('CustomerID').InvoiceDate.count().reset_index()
tx_frequency.columns = ['CustomerID','Frequency']

#add this data to our main dataframe
tx_user = pd.merge(tx_user, tx_frequency, on='CustomerID')

#plot the histogram
plot_data = [
    go.Histogram(
        x=tx_user.query('Frequency < 1000')['Frequency']
    )
]

plot_layout = go.Layout(
        title='Frequency'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
#k-means
kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Frequency']])
tx_user['FrequencyCluster'] = kmeans.predict(tx_user[['Frequency']])

#order the frequency cluster
tx_user = order_cluster('FrequencyCluster', 'Frequency',tx_user,True)

#see details of each cluster
tx_user.groupby('FrequencyCluster')['Frequency'].describe()

**Monetary**

In [ ]:
#calculate revenue for each customer
tx_uk['Revenue'] = tx_uk['UnitPrice'] * tx_uk['Quantity']
tx_revenue = tx_uk.groupby('CustomerID').Revenue.sum().reset_index()

#merge it with our main dataframe
tx_user = pd.merge(tx_user, tx_revenue, on='CustomerID')

#plot the histogram
plot_data = [
    go.Histogram(
        x=tx_user.query('Revenue < 10000')['Revenue']
    )
]

plot_layout = go.Layout(
        title='Monetary Value'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
#apply clustering
kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Revenue']])
tx_user['RevenueCluster'] = kmeans.predict(tx_user[['Revenue']])


#order the cluster numbers
tx_user = order_cluster('RevenueCluster', 'Revenue',tx_user,True)

#show details of the dataframe
tx_user.groupby('RevenueCluster')['Revenue'].describe()

In [ ]:
#calculate overall score and use mean() to see details
tx_user['OverallScore'] = tx_user['RecencyCluster'] + tx_user['FrequencyCluster'] + tx_user['RevenueCluster']
tx_user.groupby('OverallScore')['Recency','Frequency','Revenue'].mean()

In [ ]:
tx_user['Segment'] = 'Low-Value'
tx_user.loc[tx_user['OverallScore']>2,'Segment'] = 'Mid-Value' 
tx_user.loc[tx_user['OverallScore']>4,'Segment'] = 'High-Value' 



**Grafici per confronti**

In [ ]:
#Revenue vs Frequency
tx_graph = tx_user.query("Revenue < 50000 and Frequency < 2000")

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'Low-Value'")['Revenue'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'Mid-Value'")['Revenue'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['Frequency'],
        y=tx_graph.query("Segment == 'High-Value'")['Revenue'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Revenue"},
        xaxis= {'title': "Frequency"},
        title='Segments'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

#Revenue Recency

tx_graph = tx_user.query("Revenue < 50000 and Frequency < 2000")

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Low-Value'")['Revenue'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Mid-Value'")['Revenue'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['Recency'],
        y=tx_graph.query("Segment == 'High-Value'")['Revenue'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Revenue"},
        xaxis= {'title': "Recency"},
        title='Segments'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

# Revenue vs Frequency
tx_graph = tx_user.query("Revenue < 50000 and Frequency < 2000")

plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Low-Value'")['Frequency'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['Recency'],
        y=tx_graph.query("Segment == 'Mid-Value'")['Frequency'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['Recency'],
        y=tx_graph.query("Segment == 'High-Value'")['Frequency'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "Frequency"},
        xaxis= {'title': "Recency"},
        title='Segments'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)